In [1]:
#!/usr/bin/python
# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import json
import paho.mqtt.client as paho
import os
import socket
import ssl
from time import sleep
from random import uniform
import time
import datetime

In [2]:
count = 0
connflag = False

In [3]:
def on_connect(client, userdata, flags, rc):
    global connflag
    connflag = True
    print("Connection returned result: " + str(rc) )

def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))

In [4]:
mqttc = paho.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

In [5]:
host = "192.168.0.107"
port = 1883

In [6]:
mqttc.connect(host, port, keepalive=60)
mqttc.loop_start()

Connection returned result: 0


In [7]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'model_face_mask'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'model_face_mask','labelmap.pbtxt')

# Number of classes the object detector can identify
NUM_CLASSES = 2

In [8]:
## Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)


# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


In [31]:
# Initialize webcam feed
video = cv2.VideoCapture(0)
ret = video.set(3,1280)
ret = video.set(4,720)

fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0

count = 0
while(True):
    ret, frame = video.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_expanded = np.expand_dims(frame_rgb, axis=0)
    
    total_frames = total_frames + 1
    fps_end_time = datetime.datetime.now()
    dif = fps_end_time - fps_start_time
    if dif.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames/dif.seconds)
    cv2.putText(frame, str(round(fps,2)), (15,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 3,(255,20,255),3)
    cv2.putText(frame, "SSD", (550,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 3,(255,20,255),3)

    classes = 0
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: frame_expanded})
    
    k1 = np.squeeze(classes).astype(np.int32)[0]

    
    # Draw the results of the detection (aka 'visulaize the results')
    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.60) #0.60 #0.75 best

   
    cv2.imshow('Object detector', frame)
        
        
    k = int(k1) 
    #print(np.squeeze(scores)[0])
    if connflag == True and np.squeeze(scores)[0]>0.75: #and count == 5:
        tempreading = uniform(20.0,25.0)
        count=count+1
        payload = json.dumps({
                          "status":k
                          })
        mqttc.publish("Status", payload, qos=1)
        #print("msg sent: " + payload )
        count = 0
        
    if cv2.waitKey(1) == ord('q'):
        break

   # fps = int(video.get(cv2.CAP_PROP_FPS))
   # print("rate" + str(fps))
    
# Clean up
video.release()
cv2.destroyAllWindows()

In [16]:
category_index

{1: {'id': 1, 'name': 'with_mask'}, 2: {'id': 2, 'name': 'without_mask'}}